In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from hyperopt import hp
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe
import mlflow
from mlflow.models import infer_signature
import os
from mlflow import MlflowClient
import git

## MLFLOW

In [3]:
mlflow.set_tracking_uri(uri="http://localhost:5000")

## load data

In [28]:
#On recupere les données depuis les differents csv puis on les stocks dans differentes variables correspondantes (jeu de train dans X_train)

df_train = pd.read_csv("../data/archive/train.csv")
df_train
X_train, y_train = df_train["review"],  df_train["polarity"]

df_test = pd.read_csv("../data/archive/test.csv")
X_test, y_test = df_train["review"],  df_train["polarity"]
df_val = pd.read_csv("../data/archive/valid.csv")
X_val, y_val= df_train["review"],  df_train["polarity"]

Nous verifions la présence de données manquantes

In [47]:
df_train.isna().sum(), df_train.isnull().sum()

(Unnamed: 0    0
 film-url      0
 review        0
 polarity      0
 dtype: int64,
 Unnamed: 0    0
 film-url      0
 review        0
 polarity      0
 dtype: int64)

verification de la balance des labels

In [48]:
np.unique(df_train["polarity"], return_counts=True)

(array([0, 1]), array([79413, 80587]))

it's balance

## preprocessing

In [6]:
STOP_WORDS = list(
    """
a à â abord afin ah ai aie ainsi ait allaient allons
alors anterieur anterieure anterieures antérieur antérieure antérieures
apres après as assez attendu au
aupres auquel aura auraient aurait auront
aussi autre autrement autres autrui aux auxquelles auxquels avaient
avais avait avant avec avoir avons ayant

bas basee bat

c' c’ ça car ce ceci cela celle celle-ci celle-la celle-là celles celles-ci celles-la celles-là
celui celui-ci celui-la celui-là cent cependant certain certaine certaines certains certes ces
cet cette ceux ceux-ci ceux-là chacun chacune chaque chez ci cinq cinquantaine cinquante
cinquantième cinquième combien comme comment compris concernant

d' d’ da dans de debout dedans dehors deja dejà delà depuis derriere
derrière des desormais desquelles desquels dessous dessus deux deuxième
deuxièmement devant devers devra different differente differentes differents différent
différente différentes différents dire directe directement dit dite dits divers
diverse diverses dix dix-huit dix-neuf dix-sept dixième doit doivent donc dont
douze douzième du duquel durant dès déja déjà désormais

effet egalement eh elle elle-meme elle-même elles elles-memes elles-mêmes en encore
enfin entre envers environ es ès est et etaient étaient etais étais etait était
etant étant etc etre être eu eux eux-mêmes exactement excepté également

fais faisaient faisant fait facon façon feront font

gens

ha hem hep hi ho hormis hors hou houp hue hui huit huitième
hé i il ils importe

j' j’ je jusqu jusque juste

l' l’ la laisser laquelle le lequel les lesquelles lesquels leur leurs longtemps
lors lorsque lui lui-meme lui-même là lès

m' m’ ma maint maintenant mais malgre malgré me meme memes merci mes mien
mienne miennes miens mille moi moi-meme moi-même moindres moins
mon même mêmes

n' n’ na ne neanmoins neuvième ni nombreuses nombreux nos notamment
notre nous nous-mêmes nouveau nul néanmoins nôtre nôtres

o ô on ont onze onzième or ou ouias ouste outre
ouvert ouverte ouverts où

par parce parfois parle parlent parler parmi partant
pas pendant pense permet personne peu peut peuvent peux plus
plusieurs plutot plutôt possible possibles pour pourquoi
pourrais pourrait pouvait prealable precisement
premier première premièrement
pres procedant proche près préalable précisement pu puis puisque

qu' qu’ quand quant quant-à-soi quarante quatorze quatre quatre-vingt
quatrième quatrièmement que quel quelconque quelle quelles quelqu'un quelque
quelques quels qui quiconque quinze quoi quoique

relative relativement rend rendre restant reste
restent retour revoici revoila revoilà

s' s’ sa sait sans sauf se seize selon semblable semblaient
semble semblent sent sept septième sera seraient serait seront ses seul seule
seulement seuls seules si sien sienne siennes siens sinon six sixième soi soi-meme soi-même soit
soixante son sont sous souvent specifique specifiques spécifique spécifiques stop
suffisant suffisante suffit suis suit suivant suivante
suivantes suivants suivre sur surtout

t' t’ ta tant te tel telle tellement telles tels tenant tend tenir tente
tes tien tienne tiennes tiens toi toi-meme toi-même ton touchant toujours tous
tout toute toutes treize trente tres trois troisième troisièmement très
tu té

un une unes uns

va vais vas vers via vingt voici voila voilà vont vos
votre votres vous vous-mêmes vu vé vôtre vôtres

y

""".split()
)

source https://github.com/explosion/spaCy/blob/master/spacy/lang/fr/stop_words.py


we use  TfidfVectorizer, it use count vectorizer followed bytransform tfid
- count vectorizer create a matrice document x word  the cell i,j correspond of ith document avec jth word and it count the  number of occurence of the word j in i
- tfid use this matrix to calculate the tfidf matrix

### regression lineaire

In [50]:
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 42,
}

In [51]:
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(**params))
    ])# creation de la pipeline avec le preprocessing et le modèle
    

## MLFLOW EXPERIMENT

In [ ]:
## definition des commit du notebook et de la brance
session_name = os.environ.get('JPY_SESSION_NAME', '')
notebook_name = os.path.basename(session_name)
notebook_name

path = os.getcwd()
repo = git.Repo(os.path.abspath(os.path.join(path, os.pardir)))
sha_commit = repo.head.object.hexsha 
branch = repo.active_branch.name

In [ ]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow LRregression")
# Start an MLflow run
with mlflow.start_run() as run:
    mlflow.sklearn.autolog(log_datasets=False)
    pipeline.fit(X_train, y_train)
    accuracy = pipeline.score(X_test, y_test)
    mlflow.log_metric("accuracy_test", accuracy)
    mlflow.set_tag("mlflow.source.name",notebook_name)
    mlflow.set_tag("mlflow.source.git.commit", sha_commit) 
    signature = infer_signature(X_train, pipeline.predict(X_train))
    mlflow.set_tag("mlflow.source.git.branch", branch)
    mlflow.sklearn.log_model(pipeline, artifact_path="sklearn-model", signature=signature)


# Create source model version
client = MlflowClient()
src_name = "LR-staging"
client.create_registered_model(src_name)
src_uri = f"runs:/{run.info.run_id}/sklearn-model"
mv_src = client.create_model_version(src_name, src_uri, run.info.run_id)


# Copy the source model version into a new registered model
dst_name = "LR-production"
src_model_uri = f"models:/{mv_src.name}/{mv_src.version}"
mv_copy = client.copy_model_version(src_model_uri, dst_name)
print_model_version_info(mv_copy)


/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Consultez les métriques logguées par autolog lors de l'entrainement. Qu'en pensez-vous ? Ces métriques sont-elles satisfaisantes pour évaluer la qualité de votre modèle?

Ajoutez au moins une métrique pertinente d'évaluation de votre modèle.
c'est l'accuracy du training on preferait l'avoir sur le test

## build model 

In [29]:
def build_model(
    training_set,
    pipeline,
    mlflow_run_tags = None,
    mlflow_run_parameters = None,
    mlflow_run_description = None,
    validation_set = None
):
    """
    Build a sentiment analysis model, print the evaluation result and store everything to MLFlow
    @param: training_set: pandas dataframe containing the input training set
    @param: pipeline: scikit-learn pipeline that will be applied to the input data
    @param: mlflow_run_tags: dict of tags that will be stored in the MLFlow run
    @param: mlflow_run_parameters: dict of parameters that will be stored in the MLFlow run
    @param: mlflow_run_description: textual description of the run
    @param: validation_set: if provided, used to evaluate the model and log result in MLFlow
    @return: the trained pipeline
    """
    with mlflow.start_run(tags=mlflow_run_tags, description=mlflow_run_description) as run:
        if mlflow_run_parameters:
            for key, value in mlflow_run_parameters.items():
                mlflow.log_param(key, value)
        mlflow.sklearn.autolog(log_datasets=False)
        X_train, y_train = training_set["review"],  training_set["polarity"]
        pipeline.fit(X_train, y_train)
        if not validation_set.empty:
            X_val, y_val= validation_set["review"],  validation_set["polarity"]
        pipeline.score(X_val, y_val)
    return pipeline
        


In [30]:
mlflow_run_tags = {"mlflow.source.name": notebook_name,  # Replace with actual notebook name
        "mlflow.source.git.commit": sha_commit,  # Replace with actual commit SHA
        "mlflow.source.git.branch": branch  # Replace with actual branch name
    }

In [31]:
mlflow.set_experiment("MLflow LRregression C =0.1")
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=15000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(random_state=42, C = 0.1))
    ])
    
build_model(df_train, pipeline,  mlflow_run_description = "test hyper param lr c=0.1", validation_set=df_val)

/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(
2024/11/12 16:23:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run flawless-hare-249 at: http://localhost:5000/#/experiments/723041387040427611/runs/ecfae80b99b44aac9f5feb7d704e10a8.
2024/11/12 16:23:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/723041387040427611.


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=15000,
                                 stop_words=['a', 'à', 'â', 'abord', 'afin',
                                             'ah', 'ai', 'aie', 'ainsi', 'ait',
                                             'allaient', 'allons', 'alors',
                                             'anterieur', 'anterieure',
                                             'anterieures', 'antérieur',
                                             'antérieure', 'antérieures',
                                             'apres', 'après', 'as', 'assez',
                                             'attendu', 'au', 'aupres',
                                             'auquel', 'aura', 'auraient',
                                             'aurait', ...])),
                ('classifier', LogisticRegression(C=0.1, random_state=42))])

In [32]:
mlflow.set_experiment("MLflow LRregression C =2.")
pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=15000, stop_words = STOP_WORDS)),
        ('classifier', LogisticRegression(random_state=42, C = 2))
    ])
    
build_model(df_train, pipeline,  mlflow_run_description = "test hyper param lr c=2.0", validation_set=df_val)

2024/11/12 16:23:37 INFO mlflow.tracking.fluent: Experiment with name 'MLflow LRregression C =2.' does not exist. Creating a new experiment.
/Users/gigi/Desktop/mlops/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(
2024/11/12 16:24:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-asp-463 at: http://localhost:5000/#/experiments/377664617740827083/runs/9b7eaf994c864eca95e1a43f1033f570.
2024/11/12 16:24:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/377664617740827083.


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=15000,
                                 stop_words=['a', 'à', 'â', 'abord', 'afin',
                                             'ah', 'ai', 'aie', 'ainsi', 'ait',
                                             'allaient', 'allons', 'alors',
                                             'anterieur', 'anterieure',
                                             'anterieures', 'antérieur',
                                             'antérieure', 'antérieures',
                                             'apres', 'après', 'as', 'assez',
                                             'attendu', 'au', 'aupres',
                                             'auquel', 'aura', 'auraient',
                                             'aurait', ...])),
                ('classifier', LogisticRegression(C=2, random_state=42))])

## hyperopt

In [105]:
def objective(params,pipeline, training_set, validation_set,mlflow_run_description, mlflow_run_parameters, mlflow_run_tags):
    """Fonction objectif pour l'optimisation des hyperparamètres."""
    mlflow.set_experiment("MLflow LRregression opti")
    with mlflow.start_run(tags=mlflow_run_tags, description=mlflow_run_description): 
        pipeline = create_pipeline(params)
        X_train, y_train = training_set["review"],  training_set["polarity"]
        X_val, y_val= validation_set["review"],  validation_set["polarity"]
        pipeline.fit(X_train, y_train)
        
        if mlflow_run_parameters:
            for key, value in mlflow_run_parameters.items():
                mlflow.log_param(key, value)
        mlflow.sklearn.autolog(log_datasets=False)
        X_train, y_train = training_set["review"],  training_set["polarity"]
        pipeline.fit(X_train, y_train)
        mlflow.set_tag("hyperopt_candidate", True)
        y_pred = pipeline.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        if not validation_set.empty:
            X_val, y_val= validation_set["review"],  validation_set["polarity"]
        pipeline.score(X_val, y_val)
        
        return {'loss': 1 - accuracy, 'status': "ok"}


def create_pipeline(params):
    print(params)
    return Pipeline([
        ('tfidf', TfidfVectorizer(max_features=params['tfidf__max_features'])),
        ('logreg', LogisticRegression(C=params['logreg__C']))])


def build_optimized_model(training_set,
    pipeline,
    space,
    objective, 
    mlflow_run_tags = None,
    mlflow_run_parameters = None,
    mlflow_run_description = None,
    validation_set = None,
    test_set = None,
    src_name = "LR best"
    
):
    """Optimise les hyperparamètres et construit le modèle final."""
    X_train, y_train = training_set["review"],  training_set["polarity"]

    best = fmin(fn=lambda params: objective(params, pipeline, training_set, validation_set,mlflow_run_description, mlflow_run_parameters, mlflow_run_tags),
                 space=space,
                 algo=tpe.suggest,
                 max_evals=10,
                 return_argmin=False)
    
    print(f'Best parameters: {best}')


    final_model = create_pipeline(best)

    with mlflow.start_run(tags=mlflow_run_tags, description=mlflow_run_description) as run:  
        mlflow.log_params(best)
        mlflow.set_tag("hyperopt_selected", True)
        if not validation_set.empty:
            X_test, y_test= test_set["review"],  test_set["polarity"]
            final_accuracy = accuracy_score(y_test, final_model.predict(X_test))
            mlflow.log_metric("final_accuracy", final_accuracy)
    signature = infer_signature(X_train, pipeline.predict(X_train))
    mlflow.sklearn.log_model(final_model, artifact_path="sklearn-model", signature=signature)
    # Create source model version
    client = MlflowClient()
    client.create_registered_model(src_name)
    src_uri = f"runs:/{run.info.run_id}/sklearn-model"
    mv_src = client.create_model_version(src_name, src_uri, run.info.run_id)
    return final_model


In [106]:
space = {
        'tfidf__max_features': hp.choice('tfidf__max_features', [1000, 5000, 10000]),
        'logreg__C': hp.loguniform('logreg__C', np.log(0.001), np.log(100))
    }

In [ ]:
build_optimized_model(df_train, create_pipeline, space,objective,  mlflow_run_description = "test hyper param lr", validation_set=df_val, test_set=df_test)

{'logreg__C': 100, 'tfidf__max_features': 5000}                                                     
  0%|                                                         | 0/2 [00:00<?, ?trial/s, best loss=?]

2024/11/12 18:19:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run bouncy-fox-8 at: http://localhost:5000/#/experiments/158909635666735086/runs/90e6b4af30a64ca3a06c7a3de7fa08e2.

2024/11/12 18:19:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/158909635666735086.



{'logreg__C': 0.001, 'tfidf__max_features': 5000}                                                   
 50%|███████████████               | 1/2 [01:28<01:28, 88.38s/trial, best loss: 0.08079999999999998]

In [ ]:
 memory.cache(TfidfVectorizer())